In [ ]:
import sys
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../../')

from data.processing import get_data_from_source
from data.processing.processing import get_observed_dataframes
from models.seir import SIR
from main.seir.optimiser import Optimiser
from main.seir.fitting import data_setup, get_variable_param_ranges
from utils.loss import Loss_Calculator

In [ ]:
val_period=30
train_period=30
which_compartments=['total_infected']
model = SIR
num_evals = 1000
N = 13000000
initialisation = 'intermediate'
test_period = 0

In [ ]:
data = pd.read_csv('SIR_simulated.csv', header=0)
data['date'] = pd.to_datetime(data['date'])

In [ ]:
df_train = data # Or some other split

In [ ]:
# Initialise Optimiser
optimiser = Optimiser()
# Get the fixed params
if initialisation == 'starting':
    observed_values = df_district.iloc[0, :]
    start_date = df_district.iloc[0, :]['date']
    std_default_params = optimiser.init_default_params(df_train, N=N, observed_values=observed_values,
                                                       start_date=start_date, initialisation=initialisation)
elif initialisation == 'intermediate':
    std_default_params = optimiser.init_default_params(df_train, N=N, initialisation=initialisation,
                                                       train_period=train_period)

In [ ]:
if test_period == 0:
    loss_indices = [-train_period, None]
else:
    loss_indices = [-(train_period+test_period), -test_period]

In [ ]:
variable_param_ranges = {
    'lockdown_R0' : (0.3,2.3),
    'T_inf' : (2,10),
    'I_tot_ratio': (0,0.5)
}
default_params = None
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

if default_params is not None:
    default_params = {**std_default_params, **default_params}
else:
    default_params = std_default_params
default_params

In [ ]:
total_days = (df_train.iloc[-1, :]['date'] - default_params['starting_date']).days

In [ ]:
df_train

In [ ]:
default_params

In [ ]:
best_params_all, trials_all = dict(), dict()
lc = Loss_Calculator()
for i in range(20):
    best_params, trials = optimiser.bayes_opt(df_train, default_params, variable_param_ranges, model=model, 
                                              num_evals=num_evals, loss_indices=loss_indices, method='mape',
                                              total_days=total_days, which_compartments=which_compartments)
    df_prediction = optimiser.solve(best_params, default_params, df_train, end_date=df_train.iloc[-1, :]['date'],
                                        model=model)
    df_loss = lc.create_loss_dataframe_region(df_train, None, df_prediction, train_period, 
                                              which_compartments=which_compartments)
    best_params['loss'] = df_loss.loc['total_infected', 'train']
    best_params_all[i] = best_params

In [ ]:
best_params_all

In [ ]:
pd.DataFrame(best_params_all).to_csv('sir_simulated_params_loss.csv')